<img src="../demos/FIUM.png" width="350px" class="pull-right" style="display: inline-block">

# Visión Artificial

### 4º de Grado en Ingeniería Informática

Curso 2020-2021<br>
Convocatoria de Julio<br>
Marisol Isabel Zucca

In [ ]:
import math
import cv2 as cv
import numpy as np
import matplotlib.path as mpltPath

# Ejercicios

## Ejercicio 1 - CALIBRACIÓN

#### a) Realiza una calibración precisa de tu cámara mediante múltiples imágenes de un chessboard.

Para poder realizar una calibración precisa de mi cámara (en mi caso, la del móvil), vamos a apoyarnos en el siguiente chessboard y la aplicación <i>calibrate.py</i> proporcionada por el profesorado.

<img src="../images/calibracion/pattern.png" width="400px"><br>

Será necesario tomar varias fotos del marcador con distintas perspectivas, y a continuación ejecutar <i>calibrate.py</i> para que analice las fotos y obtenga la siguiente matriz:<br>

<p><i>
RMS: 3.217471748824684<br>
camera matrix:<br>
 [[3.33350026e+03 0.00000000e+00 1.37895391e+03]<br>
 [0.00000000e+00 3.30693258e+03 2.09307524e+03]<br>
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]<br>
distortion coefficients:  [ 0.21264965 -1.51913124 -0.0067358  -0.02069358  2.10661863]<br>
</i></p>

Observando el valor en la primera fila y columna, obtenemos la distancia focal precisa:

In [1]:
f = 3333.50026
f

3333.50026

#### b) Haz una calibración aproximada con un objeto de tamaño conocido y compara con el resultado anterior.

Para realizar la calibración aproximada, tomaremos una foto de un objeto de tamaño conocido a una distancia conocida. A continuación mediremos el tamaño del objeto en píxeles y aplicaremos la siguiente fórmula para obtener la distancia focal aproximada:

$$u = f \frac{X}{Z}$$

Siendo $u$ y $X$ la altura en píxeles y cm respectivamente del objeto observado, y $Z$ la distancia de la cámara, también en cm.<br>
La siguiente foto ha sido tomada a 60cm de distancia y la altura del libro es de 18cm.

<img src="../images/calibracion/b/libro.jpg" width="350px"><br>

Si recortamos la foto al filo, obtenemos un tamaño de 570x924, por lo que la altura de 18cm equivale a 924px.

<img src="../images/calibracion/b/libro18cm.jpg" width="350px"><br>

Aplicamos entonces la fórmula para obtener la distancia focal aproximada:

$$f = u \frac{Z}{X} = 924px \frac{60cm}{18cm} = 3080px$$

In [2]:
Z = 60 # distancia cámara en cm
X_book = 18 # altura en cm
u_book = 924 # altura en px

f_book = u_book * Z / X_book
f_book

3080.0

Vamos a realizar otra prueba con un segundo objeto: la distancia será la misma, pero la Nintendo Switch Lite mide 21cm en altura.

<img src="../images/calibracion/b/switch.jpg" width="350px"><br>

En este caso el valor de 21cm equivale a 1031px.

<img src="../images/calibracion/b/switch21cm.jpg" width="350px">

$$f = u \frac{Z}{X} = 1031 \frac{60cm}{21cm} = 2945.7px$$

In [3]:
Z = 60 # distancia cámara en cm
X_switch = 21 # altura en cm
u_switch = 1031 # altura en px

f_switch = u_switch * Z / X_switch
f_switch

2945.714285714286

Hemos obtenidos unos valores de distancia focal de 3080 y 2945.7 respectivamente. El valor preciso es 3333.5, que es un valor parecido ($\pm 30\%$) a los obtenidos de forma aproximada. Los resultados son, por tanto, correctos.

#### c) Determina a qué altura hay que poner la cámara para obtener una vista cenital completa de un campo de baloncesto.

Una vez más, podemos aplicar la fórmula anteriormente usada, pero teniendo la Z (la distancia de la cámara) como incógnita.
Las medidas oficiales de un campo de basket son 28 metros de largo y 15 metros de ancho.

<img src="../demos/basket.gif"><br>

Tomaremos la medida más grande como nuestra $X$. La resolución de la cámara que se está usando es de 3000 x 4000, por lo que tomaremos $u = 4000$. Finalmente, usaremos como f el valor de la calibración precisa.

$$Z = f \frac{X}{u} = 3333.5px \frac{2800cm}{4000px} = 2333.5cm$$

In [4]:
WIDTH = 3000
HEIGHT = 4000

X_basket = 28 * 100 # altura en cm
u_basket = HEIGHT # altura en px

Z_basket = f * X_basket / u_basket
Z_basket

2333.450182

Hemos obtenido un valor de 2333.5cm, es decir, para obtener una vista cenital completa de un campo de baloncesto de 28m de largo, será necesario posicionar nuestra cámara a 23,34m de altura.

#### d) Haz una aplicación para medir el ángulo que definen dos puntos marcados con el ratón en el imagen.

La aplicación implementada está basada en parte en el código de <i>medidor.py</i>, del cual se ha obtenido la forma de marcar puntos con el ratón en la imagen.

Tras marcar dos puntos en la imagen, obtendremos sus coordenadas y llamaremos a una función para aplicar la siguiente fórmula sobre los vectores obtenidos a partir de los dos puntos marcados y obtener así el ángulo que nos interesa:

$$u \times v = |u||v|cos(\alpha)$$

La función implementada es la siguiente:

In [ ]:
def angle(p1, p2):
    u = [ p1[0]-WIDTH/2, p1[1]-HEIGHT/2, f]
    v = [ p2[0]-WIDTH/2, p2[1]-HEIGHT/2, f]

    # cos(alpha) = u . v / (|u| * |v|) 
    cosAlpha = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

    return math.degrees( math.acos(cosAlpha) )

Los puntos $p1$ y $p2$ se le pasan como tuplas (x,y). A partir de estos dos puntos se obtendrán los vectores $u$ y $v$ (basándonos en el centro del imagen de ancho $WIDTH$ y alto $HEIGHT$), que son los que se usarán para, mediante el uso de funciones de las librerias numpy y math, obtener el ángulo buscado.

A continuación vemos una demostración de funcionamiento.

<img src="../demos/images/calibracion.bmp">

#### e) Opcional: determina la posición aproximada desde la que se ha tomado una foto a partir ángulos observados respecto a puntos de referencia conocidos.

## Ejercicio 2 - ACTIVIDAD

#### Construye un detector de movimiento en una región de interés de la imagen marcada manualmente. Guarda 2 ó 3 segundos de la secuencia detectada en un archivo de vídeo. Opcional: muestra el objeto seleccionado anulando el fondo.

### Implementación

En la implementación de esta aplicación se ha usado parte del código de <i>roi.py</i> para la selección de un ROI en la imagen.

La detección de movimiento en la zona delimitada por el ROI se hará mediante la comparación del frame actual con otro frame que representa al fondo.

Esta comparación entre frames se hará con la ayuda de BackgroundSubtractorMOG2, que permite la sustración del fondo de un vídeo.<br>
Como regla general, en los vídeos el fondo tiende a permanecer constante y los cambios que se detectan entre un frame y otro se deben a movimientos causados por agentes pasando por el vídeo. En alternativa al uso de BackgroundSubtractorMOG2, se podría haber guardado un frame inicial del ROI y hacer la comparación entre ese frame almacenado y el frame analizado en cada momento para detectar movimiento.

Tras guardar una imagen de fondo, otro paso fundamental para poder detectar actividad de forma eficaz, es la eliminación del ruido, ya que, de no hacerse, todos los frames serían distintos al frame de fondo por cambios mínimos en píxeles dispersos por la imagen. Por tanto, para la eliminación del ruido se ha aplicado un suavizado gaussiano a la zona del ROI.

In [ ]:
# Select ROI
[x1,y1,x2,y2] = region.roi
act_roi = frame[y1:y2+1, x1:x2+1]

# Detect activity
# Remove noise from ROI
noiseless = cv.cvtColor(act_roi, cv.COLOR_BGR2GRAY)
noiseless = cv.GaussianBlur(noiseless, (21, 21), 0)

Los siguientes pasos se harán para poder mostrar el objeto detectado anulando el fondo.
Aplicando el sustractor de fondo a la imagen sin ruido, obtendremos una máscara del objeto detectado (aplicando erosión y suavizado intentaremos mejorar los bordes de la máscara) y con esa máscara obtendremos el objeto del ROI, anulando las zona de fondo donde no aparece el objeto.

Por la forma en la que está implementado el sustractor de fondo, el objeto detectado se desvanecerá a una velocidad bastante alta en la ventana donde se muestra el mismo.

In [ ]:
# Detected activity mask
fgmask = bgsub.apply(noiseless, learningRate = -1)
fgmask = cv.erode(fgmask,kernel,iterations = 1)
fgmask = cv.medianBlur(fgmask,3)

# Detected object
obj = act_roi.copy()
obj[fgmask==0] = 0
cv.imshow('Activity', obj)

El hecho de que la mascara contenga algún valor, indica que se ha detectado algún objeto, por lo que también la usaremos para decidir si se ha detectado alguna actividad y grabar el vídeo si la opción de grabación está activa. 

In [ ]:
# Write video if any activity is detected
if fgmask.any():
    video.write(act_roi)
    putText(frame, 'Activity detected', orig=(x1,y1-8))
    if video.ON: cv.circle(act_roi,(15,15),6,(0,0,255),-1)

### Manual de uso

Para lanzar la aplicación tendremo que ejecutar el comando <b>python actividad.py</b>

Una vez inicializada, tendremos que marcar un ROI en la imagen. El ROI se puede volver a marcar las veces que sea necarios o borrar pulsando la tecla <b>x</b>.

Cuando hayamos decidido donde queremos el ROI, habrá que pulsar la tecla <b>g</b> para activar la posibilidad de grabar en vídeo.

Si se detecta alguna actividad durante la ejecución de la aplicación, aparecerá el texto "Activity detected" encima del ROI y, si la opción de grabación está activa, aparecerá un círculo rojo dentro del ROI indicando que se está grabando.

<img src="../demos/images/actividad.bmp">

Para finalizar el uso de la aplicación, volveremos a pulsar <b>g</b> y, a continuación, cerraremos la aplicación con la tecla <b>Esc</b>.

Si ha sido detectada alguna actividad durante el uso de la aplicación, se habrá generado un archivo de vídeo en el mismo directorio donde se encuentra el código. En este vídeo solo aparecerá la zona marcada por el ROI y estará compuesto unicamente por los frames en los que ha sido detectada alguna actividad.

### Video demostración

<video width="600" height="450" controls src="../demos/videos/actividad.mkv"></video>

## Ejercicio 3 - COLOR

#### Construye un clasificador de objetos en base a la similitud de los histogramas de color del ROI (de los 3 canales por separado). Opcional: Segmentación densa por reproyección de histograma.

### Implementación

En esta aplicación queremos que pasen las siguientes cosas:
* Cuando se marca un ROI con el ratón se muestran los histogramas (normalizados) de los 3 canales por separado.
* Si se pulsa una cierta tecla se guarda el recuadro como un modelo más y se muestra en la ventana "models" de la izquierda
* En todo momento (siempre que haya algún modelo guardado) se comparan los histogramas del ROI actual con los de todos los modelos.
* La menor distancia nos indica el modelo más parecido, y se muestra en la ventana "detected".

#### Dibujar el histogramas de los 3 canales por separado

Para ello se han implementado las funciones <b>calcHist()</b> y <b>DrawHistogramOnImage()</b>.

<b>calcHist()</b> se basa vagamente en el código que se ha encontrado en el siguiente repositorio de GitHub: [*This is a sample for histogram plotting for RGB images and grayscale images for better understanding of colour distribution*](https://github.com/opencv/opencv/blob/master/samples/python/hist.py). Con ella se calcula el histograma de color normalizado para un canal dado. Es necesario normalizar los histogramas ya que los modelos pueden ser rectángulos de tamaños diferentes que tampoco coinciden con el tamaño del ROI.

La función <b>DrawHistogramOnImage()</b> permite dibujar el histograma de una imagen sobre otra imagen, tras calcular el histograma de los tres canales por separado, usando la función anterior. La usaremos tanto para dibujar el histograma en el ROI mismo como para dibujarlo en una nueva ventana que nos permita verlo mejor cuando el ROI seleccionado es demasiado pequeño.

In [ ]:
def calcHist(im, channel):
    hist_item = cv.calcHist([im],[channel],None,[256],[0,256])
    cv.normalize(hist_item,hist_item,0,255,cv.NORM_MINMAX)
    return np.int32(np.around(hist_item))

def DrawHistogramOnImage(image, h_im):
    width,height,_ = h_im.shape
    for channel, color in enumerate(colors):
        hist = calcHist(image, channel)
        # Editamos los valores para que no salga boca abajo
        # y para que ocupe todo el ancho
        xs = bins*(width/256)
        ys = height-hist*(height/300)
        pts = np.int32(np.column_stack((xs,ys)))
        cv.polylines(h_im,[pts],False,color,thickness=2)

#### Guardar el modelo

Para guardar el modelo será suficiente tener una colección de algún tipo que permita añadir elementos a la misma. En este caso se ha usado un deque de 6 elementos para limitar los modelos a 6 y que se sustituyan automáticamente los más antiguos mediante una cola FIFO.

In [ ]:
models = deque(maxlen=maxlen)

##########

# Si se pulsa una cierta tecla se guarda el recuadro como un modelo
# más y se muestra en la ventana "models" de la izquierda
if key == ord('m'):
    model = roi_copy
    models.append(model)

#### Calcular las distancias de los modelos

La comparación entre histogramas se hace mediante la suma de diferencias absolutas en cada canal y quedándonos con el máximo de los tres canales.

In [ ]:
def distancia_histogramas(model, roi):
    # Suma de diferencias absolutas en cada canal
    B = np.sum( cv.absdiff( calcHist(roi, 0) , calcHist(model, 0) ) )
    G = np.sum( cv.absdiff( calcHist(roi, 1) , calcHist(model, 1) ) )
    R = np.sum( cv.absdiff( calcHist(roi, 2) , calcHist(model, 2) ) )
    # y quedarnos el máximo de los tres canales
    return max(B,G,R)

Guardaremos la distancia de cada modelo al ROI y las mostraremos arriba a la izquierda en la ventana principal.

In [ ]:
# En todo momento (siempre que haya algún modelo guardado)
# se comparan los histogramas del ROI actual con los de todos los modelos.
distances = []
for model in models:
    dist = distancia_histogramas(model, roi_copy)
    distances.append(dist/100000)

# Las distancias se muestran arriba a la izquierda
distString = ''
for dist in distances: distString += '%.2f' % dist + ' '
putText(frame, distString.strip())

#### Seleccionar el modelo más parecido

Seleccionaremos el modelo de menor distancia como el modelo detectado para el ROI y lo mostraremos en una ventana adicional "detected".

In [ ]:
if len(models) > 0:
    # Show saved models as single picture
    resized_models = [cv.resize(m, (max_h, max_h)) for m in models]
    DrawNamedWindow('models', cv.hconcat(resized_models), iX+W, iY, max_h*maxlen, max_h, normal=False)

    # Show most similar model
    detected = resized_models[distances.index(min(distances))]
    DrawNamedWindow('detected', detected, iX+W+max_h*int(maxlen/2), iY+Y+max_h, max_h*int(maxlen/2), max_h*int(maxlen/2))

### Manual de uso

Para usar la aplicación con video en vivo habrá que ejecutar el comando <b>python color.py</b>

Si queremos usar la aplicación con imágenes habrá que ejecutar el comando <b>python color.py --dev=dir:path/to/files/*</b> (ej: python color.py --dev=dir:images/color.jpg).

Si queremos usar la aplicación con videos previamente grabados habrá que ejecutar el comando <b>python color.py --dev=file:path/to/file</b> (ej: python color.py --dev=file:videos/color.mjpg).

Tras inicializar la aplicación, podremos marcar un ROI en la ventana y obtener el histograma de color de la zona marcada (aparecerá tanto en el ROI marcado como en una ventana a parte para poder verlo mejor).

Si se pulsa la tecla <b>m</b> después de marcar un ROI, se guardará la zona seleccionada como un modelo que se usará para la clasificación de objetos, comparando la similitud de los histogramas de color del modelo y del ROI.

Cuando se tenga algún modelo guardado, en las siguientes selecciones de ROI, aparecerá una ventana en la que se indique el modelo cuyo histograma de color se parecé más al del ROI actualmente seleccionado. En la zona superior izquierda de la venta principal, aparecerán los valores de distancia entre el histograma del ROI y de todos los modelos almacenados. Valores menores indican un mayor parecido y el valor menor entre todos será el modelo que se tenga como resultado detectado.

<img src="../demos/images/color.bmp">

Se pueden guardar hasta un máximo de 6 modelos. Si se guarda un séptimo modelo, se pierde el primero siguiendo una lógica FIFO (First In First Out). Pulsando la tecla <b>x</b> será posible eliminar tanto el ROI marcado, como todos los modelos almacenados.

### Video demostración

<video width="600" height="450" controls src="../demos/videos/color.mkv"></video>

## Ejercicio 4 - FILTROS

#### Muestra el efecto de diferentes filtros sobre la imagen en vivo de la webcam. Selecciona con el teclado el filtro deseado y modifica sus posibles parámetros (p.ej. el nivel de suavizado) con las teclas o con trackbars. Aplica el filtro en un ROI para comparar el resultado con el resto de la imagen. Opcional: implementa en Python o C "desde cero" algún filtro y compara la eficiencia con OpenCV.

### Implementación

Se ha decidido implementar 6 filtros:
1) Filtro de luminosidad: se aplicará una función de convolución con un kernel [0,0,0], [0,1,0], [0,0,0]<br>
2) Filtro de borde: combinando los kernels de derivada en dirección horizontal y vertical para conseguir una medida de borde en cualquier orientación.<br>
3) Filtro de visión borracha: basado en un kernel con valores en los extremos de la matriz, es decir el punto (0, 0) y el (i, i).<br>
4) Filtro box: calcula la media de un entorno de cierto radio.<br>
5) Filtro gaussiano: suavizado donde los pixels cercanos tienen más peso en el promedio.<br>
6) Filtro laplaciano: amplifica las frecuencias altas.<br>

In [ ]:
# Filtro de convolución
def cconv(im,k):
    return cv.filter2D(im,-1,k)

# Podemos combinar los kernels de derivada en dirección horizontal y
# vertical para conseguir una medida de borde en cualquier orientación
def filter_border(f_im, i=1):
    ker_hor = np.array([ [0,0,0], [-i,0,i], [0,0,0] ])
    ker_ver = ker_hor.T # np.array([ [0,-i,0], [0,0,0], [0,i,0] ])
    gx = cconv(f_im, ker_hor)
    gy = cconv(f_im, ker_ver)
    return abs(gx) + abs(gy)

# Calcula la media de un entorno de radio 5
def filter_box(f_im, kerSize=11):
    return cv.boxFilter(f_im, -1, (kerSize,kerSize))

# La forma correcta de eliminar detalles es usar el filtro gaussiano, donde los
# pixels cercanos tienen más peso en el promedio.
def filter_gaussian(f_im, kerSize=3):
    return cv.GaussianBlur(f_im, (0,0), kerSize)

# El operador Laplaciano es la suma de las segundas derivadas respecto a cada variable.
# El efecto del filtro Laplaciano es amplificar las frecuencias altas.
def filter_laplacian(f_im):
    return cv.Laplacian(f_im,-1)

Para hacer una visualización previa de los filtros seleccionados, los aplicaremos primero a la imagen completa y los mostraremos en unas ventanas extra. Tomaremos los valores de los eventuales parámetros de cada filtro de unos trackbars que se encuentran en la ventana principal.

In [ ]:
# Get values from trackbars
brightness = trackbar.Brightness / 10
border = trackbar.Border / 10
kernel = trackbar.Kernel
if kernel == 0: kernel = 1

# Define kernels for filters
ker_bright = np.array([ [0,0,0], [0,brightness,0], [0,0,0] ])

ker_drunk = np.zeros([kernel,kernel])
ker_drunk[0, 0] = 1
ker_drunk[kernel-1, kernel-1] = 1
ker_drunk = ker_drunk/np.sum(ker_drunk)

# Apply filters
filters[0] = frame
filters[1] = cconv(frame, ker_bright)
filters[2] = 3*filter_border(frame, border)
filters[3] = cconv(frame, ker_drunk)
filters[4] = filter_box(frame, kernel)
filters[5] = filter_gaussian(frame, kernel)
filters[6] = filter_laplacian(frame)

# Show filters
cv.imshow('Brightness', filters[1] )
cv.imshow('Border',     filters[2] )
cv.imshow('Drunk',      filters[3] )
cv.imshow('Box',        filters[4] )
cv.imshow('Gaussian',   filters[5] )
cv.imshow('Laplacian',  filters[6] )

Los filtros se seleccionarán mediante las teclas del 1 al 6 y se aplicarán finalmente sobre la zona indicada por el ROI.

In [ ]:
# Select ROI
[x1,y1,x2,y2] = region.roi
roi = frame[y1:y2+1, x1:x2+1]

# Select filter to apply to frame
if (key == ord('0')): opt = 0
if (key == ord('1')): opt = 1
if (key == ord('2')): opt = 2
if (key == ord('3')): opt = 3
if (key == ord('4')): opt = 4
if (key == ord('5')): opt = 5
if (key == ord('6')): opt = 6
filter = filters[opt]

# Apply filter to frame
frame[y1:y2+1, x1:x2+1] = filter[y1:y2+1, x1:x2+1]

### Manual de uso

Ejecutaremos la aplicación con el comando <b>python filtros.py</b>

Tendremos una ventana principal con 3 trackbars y 6 ventanas donde se puede tener una vista previa de los filtros.

<img src="../demos/images/filtros.bmp">

En la ventana principal será posible marcar un ROI, que será la zona donde se aplique el filtro que se seleccione a continuación. Los 6 filtros se pueden seleccionar con las teclas del 1 al 6. La tecla 0 permite quitar el filtro aplicado.

Los filtros son:<br>
1) <b>Brightness</b>: filtro de luminosidad. Se puede cambiar su intensidad con el trackbar denominado "Brightness".<br>
2) <b>Border</b>: filtro de borde. Se puede cambiar el tamaño del borde con el trackbar denominado "Border".<br>
3) <b>Drunk</b>: filtro de convolución "borracha". Se puede cambiar el tamaño del kernel con el trackbar denominado "Kernel".<br>
4) <b>Box</b>: media de un entorno de cierto radio. Se puede cambiar el tamaño del radio con el trackbar denominado "Kernel".<br>
5) <b>Gaussian</b>: suavizado Gaussiano. Se puede cambiar el nivel del suavizado con el trackbar denominado "Kernel".<br>
6) <b>Laplacian</b>: operador Laplaciano. Su efecto es amplificar las frecuencias altas.<br>

### Video demostración

<video width="600" height="450" controls src="../demos/videos/filtros.mkv"></video>

## Ejercicio 5 - SIFT

#### Escribe una aplicación de reconocimiento de objetos (p. ej. carátulas de CD, portadas de libros, cuadros de pintores, etc.) con la webcam basada en el número de coincidencias de keypoints.

### Implementación

La implementación de esta aplicación se basa en el código de <i>sift.py</i>

Como he usado Windows, me he visto obligada a usar AKAZE en lugar de xfeatures2d.<br>
También se usará BFMatcher como algoritmo de fuera bruta para encontrar asociaciones

In [ ]:
sift = cv.AKAZE_create() #sift = cv.xfeatures2d.SIFT_create(nfeatures=500)
matcher = cv.BFMatcher()

Empezaremos leyendo de un directorio las imágenes que harán de modelos para el reconocimiento de objetos mediante coincidencia de keypoints. Guardaremos la imagen, el nombre de la misma, los keypoints encontrados y los descriptores.

In [ ]:
models = []
models_names = []
models_keypoints = []
models_descriptors = []

path = Path("../images/sift")
path = path.glob("*.png")
for imagepath in path:
    image = cv.imread(str(imagepath))
    keypoints , descriptors = sift.detectAndCompute(image, mask=None)
    models.append(image)
    models_names.append(str(imagepath.stem))
    models_keypoints.append(len(keypoints))
    models_descriptors.append(descriptors)

En cada frame lo que haremos será encontrar sus keypoints y descriptores. A continuación se recorrerá la lista de modelos para encontrar las coincidencias de cada modelo. Cuando se soliciten las coincidencias de un punto, nos quedaremos con las dos mejores para poder a continuación hacer un ratio test y quedarnos solo con las coincidencias mucho mejores que las de la segunda opción encontrada.

Para comparar los modelos con el frame, nos basaremos en el porcentaje de coincidencias y no en el valor absoluto de coincidencias, ya que los modelos pueden tener diferente número de keypoints.

In [ ]:
keypoints , descriptors = sift.detectAndCompute(frame, mask=None)

bestLen = 0
secondLen = 0
index = 0

for i in range(len(models)):
    
    # solicitamos las dos mejores coincidencias de cada punto, no solo la mejor
    matches = matcher.knnMatch(descriptors, models_descriptors[i], k=2)
    
    # ratio test
    good = []
    for m in matches:
        if len(m) > 1:
            best,second = m
            if best.distance < 0.75*second.distance:
                good.append(best)

    # Si los modelos tienen diferente número de keypoints la comparación debe hacerse teniendo
    # en cuenta el porcentaje de coincidencias, no el valor absoluto.
    if len(good)/models_keypoints[i] > bestLen:
        secondLen = bestLen
        bestLen = len(good)/models_keypoints[i]
        index = i

Finalmente mostraremos por pantalla el modelo correspondiente al objeto reconocido en el caso de que el número de coincidencias sea suficiente. 

In [ ]:
if bestLen*100 > 0.5:
    # Se muestra en pequeño el modelo ganador su porcentaje, y la diferencia con el segundo mejor.
    image = cv.resize(models[index], (100, 100))
    img_height, img_width, _ = image.shape
    frame[50:50+img_height, :img_width] = image
    putText(frame, f'{1000*(t1-t0):.0f} ms {1000*(t3-t2):.0f} ms')
    putText(frame ,f'{100*bestLen:.1f} % {models_names[index]} +{100*(bestLen-secondLen):.1f} %', 
                    orig=(5,36), color=(200,255,200))

### Manual de uso

Para el utilizo de la aplicación con objetos propios, será necesario poner las fotos de los modelos en el directorio <i>../images/sift/</i>. Es recomendable el utilizo de imágenes de resolución reducida, ya que de lo contrario la aplicación sufrirá mucho lag.

La aplicación se inicializará ejecutando <b>python sift.py</b>

Cuando la aplicación esté funcionando, será suficiente enfocar el objeto que se quiera reconocer para que se detecten los keypoints del mismo y se reconozca como el modelo de objeto con el que comparta más coincidencias. El objeto podrá estar rotado o salirse en parte del encuadre de la cámara y seguirá siendo reconocido.

<img src="../demos/images/sift.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/sift.mkv"></video>

## Ejercicio 6 - RECTIF

#### Rectifica la imagen de un plano para medir distancias (tomando manualmente referencias conocidas). Por ejemplo, mide la distancia entre las monedas en coins.png o la distancia a la que se realiza el disparo en gol-eder.png. Verifica los resultados con imágenes originales tomadas por ti.

### Implementación

### Manual de uso

<img src="../demos/images/rectif.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/rectif.mkv"></video>

## Ejercicio 7 - PANO

#### Crea automáticamente un mosaico a partir de las imágenes en una carpeta. Las imágenes no tienen por qué estar ordenadas ni formar una cadena lineal y no sabemos el espacio que ocupa el resultado. El usuario debe intervenir lo menos posible. Recuerda que debe tratarse de una escena plana o de una escena cualquiera vista desde el mismo centro de proyección. Debes usar homografías. Compara el resultado con el que obtiene la utilidad de stitching de OpenCV.

### Implementación

Al igual que se hizo en el ejercicio 5 SIFT, vamos a hacer uso de AKAZE y BFMatcher para poder encontrar las correspondencias de keypoints entre dos imágenes. Se hará uso de la función <b>match()</b> que viene implementada en el notebook <i>transf2D.ipyng</i>.

In [ ]:
def match(query, model):

    x1 = cv.cvtColor(query, cv.COLOR_BGR2GRAY)
    x2 = cv.cvtColor(model, cv.COLOR_BGR2GRAY)

    (k1, d1) = sift.detectAndCompute(x1, None)
    (k2, d2) = sift.detectAndCompute(x2, None)
    
    matches = bf.knnMatch(d1,d2,k=2)

    # Apply ratio test
    good = []
    for m in matches:
        if len(m) == 2:
            best, second = m
            if best.distance < 0.75*second.distance:
                good.append(best)
    
    # findHomography doesn't work with less than 4 points
    if len(good) < 4: return len(good), None
    
    # a partir de los matchings seleccionados construimos los arrays de puntos que necesita findHomography
    src_pts = np.array([ k2[m.trainIdx].pt for m in good ]).astype(np.float32).reshape(-1,2)
    dst_pts = np.array([ k1[m.queryIdx].pt for m in good ]).astype(np.float32).reshape(-1,2)
    
    H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 3) # cv.LMEDS
    
    # mask viene como una array 2D de 0 ó 1, lo convertimos a un array 1D de bool
    return sum(mask.flatten() > 0), H

Tras leer las imágenes que forman parte de una foto panorámica, obtendremos los números de coincidencias de cada foto con cada una de las otras fotos usando la función anterior. Esta función nos devolverá tuplas conteniendo el número de matches y la pareja de imágenes a la que se refiere ese número de matches. Por ejemplo un valor devuelto de (52, 5, 6) indica que las imágenes 5 y 6 tienen 52 correspondencias de keypoints.

A continuación ordenaremos los matches encontrados de mayor a menor y eliminaremos los que están por debajo de cierto umbral.

In [ ]:
dirPath = '../images/pano/my_scene/'

threshold = 4

# Load the images
pano = [cv.imread(x) for x in sorted( glob.glob(dirPath+'*.jpg') )]

# Sort matched images by number of matching points
sortedMatches = sorted([(match(p,q)[0],i,j) for i,p in enumerate(pano) for j,q in enumerate(pano) if i< j],reverse=True)

# Remove those below a certain threshold
sortedMatches = [s for s in sortedMatches if s[0] >= threshold]

Para encontrar la imagen central, recorreremos la lista de matches resultante de las operaciones anteriores e iremos sumando los matches que cada imagen tiene con las demás. Si tomamos como ejemplo el match (52, 5, 6), sumaremos el valor 52 tanto al contador de matches de la imagen 5 como al de la imagen 6. La imagen que termine teniendo el valor total de matches mayor, será la que se tome como centro.

In [ ]:
# Find the center image (the one that matches with more pictures)
maxMatches = [0] * len(pano)
for sm in sortedMatches:
    matches, fst, snd = sm
    maxMatches[fst] += matches
    maxMatches[snd] += matches

center = maxMatches.index(max(maxMatches))

En el paso siguiente lo que haremos será recorrer en bucle la lista de matches para ir calculando y aplicando las homografías entre una imagen que ya forma parte de la imagen resultado y otra cuya perspectiva aún no haya sido transformada.

En los primeros recorridos del bucle, lo que se hará será buscar imágenes que tengan coincidencias con la imagen tomada como centro, obteniendo la homografía para transformar la imagen no central a la perspectiva de la imagen central. En los sucesivos recorridos se buscarán imágenes que tengan coincidencias tanto con la imagen central como con las que ya han sido transformadas. En este segundo caso, la homografía que se tendrá que aplicar es una composición entre la homografía de la imagen transformada no central y la imagen que se ha encontrado tener coincidencias con la imagen ya transformada.

Conforme vamos recorriendo el bucle, guardaremos las homografías que encontremos y también las esquinas de las imágenes transformadas.

In [ ]:
def transform_corners(H, img):
    h,w,_ = img.shape
    corners = np.array([ [0,0],[0,h],[w,h],[w,0] ])
    trans_corners = htrans(H, corners)
    xx = [x for x,_ in trans_corners]
    yy = [y for _,y in trans_corners]
    return min(xx), max(xx), min(yy), max(yy)

In [ ]:
# Homography array to store homographies for composition
homographies = [None] * len(pano)

# Array to check if an image was already used
used = [False] * len(pano)
used[center] = True

# Array to store the min/max corners values of the images
corners = [None] * len(pano)
h,w,_ = pano[center].shape
corners[center] = (0, w, 0, h)

# While there are still images to put in the result
while len(sortedMatches) > 0 and not all(used):
    
    for sm in sortedMatches:
        _, fst, snd = sm
        
        # Check if one of the images is already in the result
        # and match it with one that is not 
        if used[fst] and not used[snd]:
            x1 = fst
            x2 = snd
        elif used[snd] and not used[fst]:
            x1 = snd
            x2 = fst
        else: continue

        print('Matching '+str(x1)+' and '+str(x2))

        _,H = match(pano[x1],pano[x2])

        # Composite the homography if the image is not directly
        # connected to the center image
        if homographies[x1] is not None:
            H = homographies[x1]@H

        # Mark the image as used
        used[x2] = True

        # Save the homography for future compositions
        homographies[x2] = H

        # Save the corners
        corners[x2] = transform_corners(H, pano[x2])
    
    # Remove matches that we don't need anymore
    sortedMatches = [s for s in sortedMatches if not (used[s[1]] and used[s[2]]) ]

La razón por la que se guardan los valores <b>x</b> e <b>y</b> de las esquinas de las imágenes transformadas es para poder encontrar a continuación los valores menores y mayores absolutos de <b>x</b> e <b>y</b> que usaremos para determinar el tamaño y desplazamientos adecuados para poder contener de manera ajustada la imagen resultante de la transformación de todas las imágenes en una única foto panorámica.

In [ ]:
# Set sizes for the result image
xmin = int( math.ceil( min([t[0] for t in corners if t is not None]) ) )
xmax = int( math.ceil( max([t[1] for t in corners if t is not None]) ) )
ymin = int( math.ceil( min([t[2] for t in corners if t is not None]) ) )
ymax = int( math.ceil( max([t[3] for t in corners if t is not None]) ) )

width = xmax - xmin
height = ymax - ymin

T = desp((-xmin, -ymin))
size = (width, height)

Tras haber obtenido la anchura, altura y desplazamiento correctos, podemos finalmente aplicar las homografías encontradas anteriormente y transformar realmente las imágenes.

In [ ]:
# Put the center image in the result
result = cv.warpPerspective(pano[center], T , size)

# Put the rest of the images in the result
for i, H in enumerate(homographies):
    if H is not None:
        cv.warpPerspective(pano[i], T@H, size, result, 0, cv.BORDER_TRANSPARENT)

Los resultados que se obtienen no son perfectos ya que en parte dependen del uso de AKAZE frente a xfeatures2d, lo que encuentra un número menor de coincidencias entre imágenes.

<img src="../demos/images/pano_my.jpg"><br>

Podemos ver que la imagen se transforma con una perspectiva correcta, pero presenta varios errores de alineamiento, por ejemplo en los bolígrafos o en las separaciones entre baldosas.

Otro resultado cuestionable se obtiene usando las imágenes de ejemplo que se pueden encontrar en los notebooks de la asignatura.

<img src="../demos/images/pano_fium.jpg"><br>

Vemos como la reconstrucción se hace bien desde el centro hacia el lado derecho, pero el resto de la imagen continua boca abajo y desde el lado izquierdo hacia el centro. Esto se debe al hecho de que en el cálculo de la imagen central, se toma la imagen 6 (la segunda de la esquina izquierda) como referencia, lo que hace que la transformación de perspectiva se vaya haciendo siempre más esagerada. La foto ha sido tomada rotando la camara del lado izquierdo al lado derecho y termina teniendo forma de anfiteatro

Si seleccionaramos manualmente la cuarta imágen como imagen central, se obtendría un resultado correcto en todos los aspectos.

<img src="../demos/images/pano_fium2.jpg"><br>

### Manual de uso

Si queremos probar la aplicación con otras imágenes podemos modificar el contenido del directorio <i>../images/pano/my_scene/</i> o editar la línea de código <b>dirPath = '../images/pano/my_scene/'</b> para que indique el directorio desde el que queremos leer las imágenes.

A continuación solo tendremos que ejecutar el comando <b>python pano.py</b> y esperar el resultado. Cuando se empiece a hacer matching entre las fotos, empezarán a aparecer por consola las parejas de imágenes que se van juntando mediante transformaciones de perspectiva. Cuando se termine de generar la foto, nos aparecerá su resultado en una ventana de matplotlib. También se guardará la imagen resultado en el subdirectorio <i>results</i> que se crea en el directorio donde se encuentran las imágenes.

### Video demostración

<video width="600" height="450" controls src="../demos/videos/pano.mkv"></video>

## Ejercicio 8 - RA

#### Crea un efecto de realidad aumentada interactivo.

El código de este ejercicio está basado en la aplicación <i>pose2.py</i>

#### a) Los objetos virtuales deben cambiar de forma, posición o tamaño siguiendo alguna lógica
Cuando se encuentre un marcador, se dibujará un cubo transparente sobre el mismo, que se irá moviendo recorriendo los lados del cuadrado definido por las equinas más largas del marcador. Pulsando las tecla <b>+</b> y <b>-</b> podremos cambiar el tamaño del cubo.

#### b) El usuario puede observar la escena cambiante desde cualquier punto de vista moviendo la cámara alrededor del marcador
Esta parte del ejercicio viene implementada por el código que ya se encuentra en <i>pose2.py</i>, por lo que no ha sido necesario implementarlo personalmente.

#### c) El usuario puede marcar con el ratón en la imagen puntos del plano de la escena para interactuar con los objetos virtuales.
Haciendo click sobre el cubo en movimiento, se podrá activar o desactivar el texturizado del mismo.

Las funciones de interés que permiten el funcionamiento indicado se detallan a continuación.

### Implementación

El movimiento del cubo se hace a través del uso de las funciones seno y coseno, que permiten obtener valores entre -1 y 1 a lo largo del tiempo (dado en número de frames).

Usaremos los valores del seno para determinar la dirección en la que tenga que moverse el cubo sobre el marcador.
* Cuando el valor del seno esté creciendo del 0 al 1, el cubo tendrá que moverse hacia arriba.
* Cuando el valor del seno esté decreciendo del 1 al 0, el cubo tendrá que moverse hacia la derecha.
* Cuando el valor del seno esté decreciendo del 0 al -1, el cubo tendrá que moverse hacia abajo.
* Cuando el valor del seno esté creciendo del -1 al 0, el cubo tendrá que moverse hacia la izquierda.

In [ ]:
UD = 0 # up/down movement
LR = 0 # left/right movement
sin = 0
cos = 1
minSize = 0.25 # minimum size of the cube
maxSize = 1    # maximum size of the cube
size = minSize # actual size of the cube

pikapika = False # if True, draw Pikachu

#########################################
# Lo que sigue va en el bucle del video #
#########################################

# Move the cube along the marker
oldsin = sin
sin = np.sin(n/50)

if oldsin <= sin and sin >= 0 and sin <= 1: # up
    UD = abs(sin)
    LR = 0
elif oldsin > sin and sin >= 0 and sin <= 1: # right
    UD = 1
    LR = abs(np.cos(n/50))
elif oldsin >= sin and sin <= 0 and sin >= -1: # down
    UD = abs(np.cos(n/50))
    LR = 1
elif oldsin < sin and sin <= 0 and sin >= -1: # left
    UD = 0
    LR = abs(sin)

El control sobre si se ha hecho click sobre el cubo se hará transformando los puntos 3D del cubo en puntos 2D de la imagen y comprobando si las coordenadas (x,y) del punto marcado se encuentran dentro de las coordenadas del polígono 2D que se dibuja. Esta comprobación se hace con la función <b>contains_points()</b> de la librería <b>matplotlib.path</b>.

Cuando se hace click sobre el cubo, se pasa de un cubo transparente a un cubo texturizado y de uno texturizado a uno transparente.

In [ ]:
fig3D = cube*size + ((1-size)*LR, (1-size)*UD, 0)
fig2D = htrans(M, fig3D).astype(int)

# Check if the clicked point is inside the figure
if points[0] != (0,0):
    if mpltPath.Path(fig2D).contains_points(points):
        pikapika = not pikapika
    points[0] = (0,0)

if pikapika:
    # If the cube was clicked, draw surprised Pikachu
    drawPikachu(fig2D, frame)
else:
    # Otherwise draw a wire cube
    cv.drawContours(frame, [fig2D], -1, (0,128,0), 3, cv.LINE_AA)

La función <b>drawPikachu()</b> permite darle textura al cubo cuando se haga click sobre el mismo.
Según la perspectiva con la que se observe, de un cubo sólido será posible ver 1, 2 o como mucho 3 caras a la vez. Para que el texturizado del cubo salga correctamente, nos interesa que las caras visibles se dibujen por últimas.

Por la forma en la que funciona la detección del marcador, podemos asegurar que el cubo nunca podrá verse desde una perspectiva inferior, por lo que podemos basarnos en el valor de la $x$ de la esquina inferior izquierda de cada lado para determinar si se encuentra delante o detrás de otro lado. Un valor más alto de $x$ indica que la cara del cubo se encuentra delante de otra con un valor de $x$ menor.

Tras determinar las caras del cubo, lo que haremos será ordenarlas según el valor de $x$ de la esquina indicada anteriormente, lo que nos permitirá dibujar primero las que se encuentran detrás.

Aplicaremos <b>warpPerspective()</b> con la homografía definida por los puntos de la cara que se quiera dibujar para transformar la perspectiva de la imagen de Pikachu.

In [ ]:
pikachu = cv.imread('../images/ra/pikachu.png')
h,w = pikachu.shape[:2]
src = np.array([[0,0],[0,h],[w,h],[w,0]])

def drawPikachu(cube, frame):
    sides = [
        ( cube[5][0], np.array([cube[5], cube[0], cube[1], cube[6]]) ),
        ( cube[6][0], np.array([cube[6], cube[1], cube[2], cube[7]]) ),
        ( cube[7][0], np.array([cube[7], cube[2], cube[3], cube[8]]) ),
        ( cube[8][0], np.array([cube[8], cube[3], cube[0], cube[5]]) )
    ]

    # A max of 3 sides can be seen at any given time and the cube will
    # never appear from below. The 2 sides with the highest value of x,
    # will be the ones that are actually seen. Draw them last.
    sides.sort(key=lambda s : s[0], reverse=True)

    # Top side must be the last to be drawn
    sides.append( (cube[5][0], cube[5:9]) )

    for _, dst in sides:
        H = cv.getPerspectiveTransform(src.astype(np.float32), dst.astype(np.float32))
        cv.warpPerspective(pikachu,H,(WIDTH,HEIGHT),frame,0,cv.BORDER_TRANSPARENT)

### Manual de uso

Para el uso de la aplicación será necesario tener el siguiente marcador en forma de L.

<img src="../images/ra/ref.png">

Colocaremos el marcador en la escena que nos interese y lo enfocaremos con la camara. A continuación inicializaremos la aplicación con <b>python ra.py</b>

Con las teclas <b>+</b> y <b>-</b> podremos cambiar el tamaño del cubo, y haciendo click sobre el mismo podremos activar y desactivar el texturizado.

<img src="../demos/images/ra01.bmp">
<img src="../demos/images/ra02.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/ra.mkv"></video>

## Ejercicio 9 - SWAP

#### Intercambia dos cuadriláteros en una escena marcando manualmente los puntos de referencia.

### Implementación

Para la implementación de esta aplicación se ha vuelto a hacer uso del código <i>medidor.py</i> para la selección de puntos con el ratón.

Las funciones que permiten el funcionamiento de la aplicación son <b>mask_zone()</b> y <b>copy_quad()</b>.

Para la implementación de <b>mask_zone()</b> se ha consultado la siguiente entrada en Stack Overflow: [*Extracting polygon given coordinates from an image using OpenCV*](https://stackoverflow.com/questions/30901019/extracting-polygon-given-coordinates-from-an-image-using-opencv). Esta función permite generar una máscara poligonal a partir de ciertos puntos dados. Esa máscara se aplicará sobre la imagen original para obtener así una imagen en la que solo aparece la parte contenida en el polígono dado.

In [ ]:
def mask_zone(points):
    mask = np.zeros((frame.shape[0], frame.shape[1]))
    cv.fillConvexPoly(mask, points, 1)
    mask = mask.astype(np.bool)
    out = np.zeros_like(frame)
    out[mask] = frame[mask]
    return out, mask

La función <b>copy_quad()</b> recibirá dos cuadriláteros <b>p</b> y <b>q</b> definidos como 4 puntos cada uno. Generará una mascara para cada uno de ellos y la imagen obtenida tras aplicar la respectiva máscara sobre la imágen original.

A continuación hará una transformación de perspectiva de los puntos de <b>p</b> a los de <b>q</b> y viceversa, obteniendo las dos homografías. Con las homografías obtenidas podremos transformar la imagen contenida en <b>p</b> para que se adapte al espacio definido por <b>q</b> y la imagen contenida en <b>q</b> para que se adapte al espacio definido por <b>p</b>.

Finalmente aplicamos las dos máscaras sobre la imagen original para intercambiar de sitio las dos imágenes transformadas.

In [ ]:
def copy_quad(p, q, frame):

    src, mask1 = mask_zone(p)
    dst, mask2 = mask_zone(q)

    Hpq = cv.getPerspectiveTransform(p.astype(np.float32), q.astype(np.float32))
    Hqp = cv.getPerspectiveTransform(q.astype(np.float32), p.astype(np.float32))

    out1 = frame.copy()
    out2 = frame.copy()

    h,w,_ = frame.shape

    cv.warpPerspective(src,Hpq,(w,h),out1,0,cv.BORDER_TRANSPARENT)
    cv.warpPerspective(dst,Hqp,(w,h),out2,0,cv.BORDER_TRANSPARENT)

    frame[mask2] = out1[mask2]
    frame[mask1] = out2[mask1]

### Manual de uso

Para el uso correcto de la aplicación, se deberán seleccionar un total de 8 puntos, 4 por cada cuadrilatero. El primer punto por el que se empiece no es relevante, pero el orden en el que se marquen los siguientes puntos de cada cuadrilátero deberá ser coherente para ambos.

Por ejemplo, si se empieza por la esquina superior izquierda, habrá que seguir con la otra esquina superior o la inferior izquierda, y seguir seleccionando los puntos que permitan dibujar un cuadrilátero. Los puntos del segundo cuadrilátero deberán ser seleccionados en el mismo orden que los del primero.

<img src="../demos/images/swap01.bmp"><br>

Cuando se hayan marcado los 8 puntos, se intercambiarán las imágenes contenidas en ellos, aplicándole transformaciones para que se adapten al nuevo espacio. La calidad del resultado de los bordes depende del cuidado con el que se seleccionen los 8 puntos.

<img src="../demos/images/swap02.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/swap.mkv"></video>

# Bibliografía

[*This is a sample for histogram plotting for RGB images and grayscale images for better understanding of colour distribution*](https://github.com/opencv/opencv/blob/master/samples/python/hist.py) - Abid Rahman

[*Extracting polygon given coordinates from an image using OpenCV*](https://stackoverflow.com/questions/30901019/extracting-polygon-given-coordinates-from-an-image-using-opencv) - Stack Overflow